# Cleaning Congressional Trading Data
Download price data from yfinance for relevant tickers

In [ ]:
import yfinance as yf
import pandas as pd

# Read tickers from file
with open('MAR', 'r') as file:
    tickers = [line.strip() for line in file.readlines()]

all_returns = {}
for ticker in tickers:
    try:
        stock_data = yf.Ticker(ticker).history(period='1d', start='2018-01-01')
        # Check if the data is empty
        if stock_data.empty:
            raise ValueError('No data found')
        daily_returns = stock_data['Close'].dropna()
        all_returns[ticker] = daily_returns
        print(f"Acquired information for {ticker}")
    except Exception as e:
        print(f"Error with ticker {ticker}: {str(e)}")
        continue

returns_df = pd.DataFrame(all_returns)



Clean returns data by concatenating


In [ ]:
# Parse the 'Date' column to remove the time component, keeping only the date
returns_df['Date'] = pd.to_datetime(returns_df['Date']).dt.date

# Group the data by date and sum up the values for each column
df_grouped = returns_df.groupby('Date').sum().reset_index()

# Display the first few rows of the grouped DataFrame
df_grouped.head()

Save File

In [ ]:
# Save DF as CSV file
df_grouped_rest = returns_df.reset_index()
df_grouped_rest.to_csv('returns_with_dates_and_tickers.csv', index=False)

## Join Trading Data with Sentiment
Use Congress Trading file along with wsb-all (Wall Street Bet's Sentiment Score) to match sentiment based on ticker and date to then further explore relationships around sentiment

In [3]:
import pandas as pd

# Load the Congress trading data and WSB data into DataFrames
df_congress = pd.read_excel('/Users/lukejohnson/Desktop/Congressional Trading/congress-trading-all.xlsx')
df_wsb = pd.read_excel('/Users/lukejohnson/Desktop/Congressional Trading/wsb-all.xlsx')

# Display the first few rows of each DataFrame to understand their structures
df_congress.head(), df_wsb.head()

# Parse the date columns to ensure they are in datetime format
df_congress['TransactionDate'] = pd.to_datetime(df_congress['TransactionDate'])
df_wsb['Datetime'] = pd.to_datetime(df_wsb['Datetime'])

# Merge the DataFrames based on 'TransactionDate' and 'Ticker'
df_merged = pd.merge(df_congress, df_wsb, left_on=['TransactionDate', 'Ticker'], right_on=['Datetime', 'Ticker'], how='left')

# Create a new DataFrame that includes only the relevant columns from the merged DataFrame
df_final = df_merged[['ReportDate', 'TransactionDate', 'Ticker', 'Representative', 'Transaction', 'Amount', 'Party', 'House', 'Range', 'Sentiment']]

# Display the first few rows of the final DataFrame
df_final.head()

,ReportDate,TransactionDate,Ticker,Representative,Transaction,Amount,Party,House,Range,Sentiment
0,2023-08-17,2023-08-15,ARCC,Pete Sessions,Purchase,155.92,R,Representatives,$155.92,NaN
1,2023-08-17,2023-07-28,KD,Pete Sessions,Sale,163.20,R,Representatives,$163.20,NaN
2,2023-08-16,2023-07-31,JPM,Kevin Hern,Purchase,1001.00,R,Representatives,"$1,001-$15,000",0.074633
3,2023-08-14,2023-07-25,MMM,Tommy Tuberville,Purchase,1001.00,R,Senate,"$1,001 - $15,000",0.074200
4,2023-08-14,2023-07-13,LPX,Tommy Tuberville,Sale,1001.00,R,Senate,"$1,001 - $15,000",NaN


Count number of sentiment values that we have in our data to determine viability of sentiment scores to find anomalies.

In [4]:
# Count the number of non-NaN values in the 'Sentiment' column
non_nan_sentiment_count = df_final['Sentiment'].count()

non_nan_sentiment_count

1637

## Merging Price Data with Congress Data

In [8]:
# Load the returns data that includes stock prices
df_returns = pd.read_csv('/Users/lukejohnson/Desktop/Congressional Trading/returns.csv')

# Display the first few rows of each DataFrame to understand their structures
df_final.head(), df_returns.head()

# Parse the date columns to ensure they are in datetime format
df_final.loc[:, 'TransactionDate'] = pd.to_datetime(df_final['TransactionDate'])
df_returns['Date'] = pd.to_datetime(df_returns['Date'], format='%m/%d/%y')
df_returns['Date'] = pd.to_datetime(df_returns['Date'])

# Melt the returns DataFrame to make it long-form, which will make the merge easier
df_returns_melted = df_returns.melt(id_vars=['Date'], var_name='Ticker', value_name='Price')

# Merge the DataFrames based on 'TransactionDate' and 'Ticker'
df_final_with_price = pd.merge(df_final, df_returns_melted, left_on=['TransactionDate', 'Ticker'], right_on=['Date', 'Ticker'], how='left')

# Drop the additional 'Date' column from the merge
df_final_with_price.drop(columns=['Date'], inplace=True)

# Display the first few rows of the final DataFrame with price
df_final_with_price.head()


,ReportDate,TransactionDate,Ticker,Representative,Transaction,Amount,Party,House,Range,Sentiment,Price
0,2023-08-17,2023-08-15,ARCC,Pete Sessions,Purchase,155.92,R,Representatives,$155.92,NaN,19.350000
1,2023-08-17,2023-07-28,KD,Pete Sessions,Sale,163.20,R,Representatives,$163.20,NaN,13.220000
2,2023-08-16,2023-07-31,JPM,Kevin Hern,Purchase,1001.00,R,Representatives,"$1,001-$15,000",0.074633,157.960007
3,2023-08-14,2023-07-25,MMM,Tommy Tuberville,Purchase,1001.00,R,Senate,"$1,001 - $15,000",0.074200,108.211685
4,2023-08-14,2023-07-13,LPX,Tommy Tuberville,Sale,1001.00,R,Senate,"$1,001 - $15,000",NaN,78.125320


How much price information were we able to merge?

In [9]:
# Count the number of non-NaN values in the 'Price' column
non_nan_price_count = df_final_with_price['Price'].count()

non_nan_price_count

25641

In [12]:
# Save the final DataFrame as a CSV file
csv_path_final = '/Users/lukejohnson/Desktop/Congressional Trading/congress_main.csv'
df_final_with_price.to_csv(csv_path_final, index=False)

csv_path_final


'/Users/lukejohnson/Desktop/Congressional Trading/congress_main.csv'

## Low End and High End
Create Low End and High End values of portfolio to assume that an average over a large sample eize will be fairly representative of portfolio value. We want to understand their overall portfolio and to do so we need to model out their porfolio based on the trades they have made. Once we understand what the portfolio value is and how they are perrforming on different postions we can further determine where they are making moves out of the ordianry. 

In [13]:
# Remove dollar signs and commas
df_final_with_price['Range'] = df_final_with_price['Range'].str.replace('$', '').str.replace(',', '')

# Split the 'Range' into two columns: 'Low_End' and 'High_End'
df_final_with_price[['Low_End', 'High_End']] = df_final_with_price['Range'].str.split('-', expand=True)

# Convert the new columns to numeric form
df_final_with_price['Low_End'] = pd.to_numeric(df_final_with_price['Low_End'], errors='coerce')
df_final_with_price['High_End'] = pd.to_numeric(df_final_with_price['High_End'], errors='coerce')

# Show the first few rows to verify the changes
df_final_with_price[['Range', 'Low_End', 'High_End']].head()


,Range,Low_End,High_End
0,155.92,155.92,NaN
1,163.20,163.20,NaN
2,1001-15000,1001.00,15000.0
3,1001 - 15000,1001.00,15000.0
4,1001 - 15000,1001.00,15000.0


In [36]:
# Create new columns to store the adjusted transaction values for 'High_End' and 'Low_End'
df_final_with_price['Adjusted_Low_End'] = df_final_with_price.apply(lambda row: -row['Low_End'] if row['Transaction'] == 'Purchase' else row['Low_End'], axis=1)
df_final_with_price['Adjusted_High_End'] = df_final_with_price.apply(lambda row: -row['High_End'] if row['Transaction'] == 'Purchase' else row['High_End'], axis=1)

# Group by 'Representative' to calculate the portfolio value for both 'Adjusted_Low_End' and 'Adjusted_High_End'
portfolio_low_end = df_final_with_price.groupby('Representative')['Adjusted_Low_End'].sum().reset_index()
portfolio_high_end = df_final_with_price.groupby('Representative')['Adjusted_High_End'].sum().reset_index()

# Merge the two DataFrames to have both 'Low_End' and 'High_End' portfolio values in a single DataFrame
portfolio_summary = pd.merge(portfolio_low_end, portfolio_high_end, on='Representative', how='inner')
portfolio_summary.columns = ['Representative', 'Portfolio_Low_End', 'Portfolio_High_End']

df_final_with_price.head()

# Sort the DataFrame by 'Representative' and 'TransactionDate' for calculating the running portfolio value
df_main_sorted = df_final_with_price.sort_values(by=['Representative', 'TransactionDate'])

# Calculate the running total for both 'Adjusted_Low_End' and 'Adjusted_High_End' for each 'Representative'
df_main_sorted['Running_Portfolio_Low_End'] = df_main_sorted.groupby('Representative')['Adjusted_Low_End'].cumsum()
df_main_sorted['Running_Portfolio_High_End'] = df_main_sorted.groupby('Representative')['Adjusted_High_End'].cumsum()

df_main_sorted['Est_Portfolio_Value'] = (df_main_sorted['Running_Portfolio_High_End'] + df_main_sorted['Running_Portfolio_Low_End'])  / 2

# Show the first few rows to verify the changes
df_main_sorted[['Representative', 'TransactionDate', 'Adjusted_Low_End', 'Adjusted_High_End', 'Running_Portfolio_Low_End', 'Running_Portfolio_High_End', 'Est_Portfolio_Value']].head()


,Representative,TransactionDate,Adjusted_Low_End,Adjusted_High_End,Running_Portfolio_Low_End,Running_Portfolio_High_End,Est_Portfolio_Value
28043,A. Mitchell Jr. McConnell,2017-09-07,-1001.0,-15000.0,-1001.0,-15000.0,-8000.5
28044,A. Mitchell Jr. McConnell,2017-12-05,-1001.0,-15000.0,-2002.0,-30000.0,-16001.0
28054,A. Mitchell Jr. McConnell,2018-03-05,-1001.0,-15000.0,-3003.0,-45000.0,-24001.5
27828,A. Mitchell Jr. McConnell,2018-06-01,-1001.0,-15000.0,-4004.0,-60000.0,-32002.0
26691,A. Mitchell Jr. McConnell,2018-09-06,-1001.0,-15000.0,-5005.0,-75000.0,-40002.5


## Further Clean
### This gives us 3 data frames that show us the average amount owned of every security as of today.
This information is not the most valuable thing to us as of now but it does allow us to continue to build out the portfolio's of each represenative.

we want to remove all Price 0 this edits our data from 31,000 rows to 26,000 rows which is still a large sample. We then want to caculate High end share count and low end share count by taking adjusted high end or low end of range divided by price. Then we take the unique tickers for each representative and we then index based upon representatives and denote the assets they own in columns in order to then track portfolio value. THen by taking low end and high end we have average value and over a large sample size we hope this to be representative of portfolio value. 

In [37]:
# Remove rows where the 'Price' column is zero or NaN
df_main_filtered = df_main_sorted[df_main_sorted['Price'] > 0].copy()

# Calculate the number of shares sold or purchased for both high-end and low-end transaction values
# Using the formula: Shares = Transaction Value / Price
df_main_filtered['Shares_High_End'] = df_main_filtered['Adjusted_High_End'] / df_main_filtered['Price']
df_main_filtered['Shares_Low_End'] = df_main_filtered['Adjusted_Low_End'] / df_main_filtered['Price']

# Create separate DataFrames for the quantity of assets held by each representative
# Initialize them with zeros
unique_reps = df_main_filtered['Representative'].unique()
unique_assets = df_main_filtered['Ticker'].unique()

df_shares_high_end = pd.DataFrame(0, index=unique_reps, columns=unique_assets)
df_shares_low_end = pd.DataFrame(0, index=unique_reps, columns=unique_assets)

# Update the DataFrames to reflect the transactions
for _, row in df_main_filtered.iterrows():
    rep = row['Representative']
    asset = row['Ticker']
    if row['Transaction'] == 'Purchase':
        df_shares_high_end.at[rep, asset] += row['Shares_High_End']
        df_shares_low_end.at[rep, asset] += row['Shares_Low_End']
    else:  # For 'Sale'
        df_shares_high_end.at[rep, asset] -= row['Shares_High_End']
        df_shares_low_end.at[rep, asset] -= row['Shares_Low_End']

# Calculate the average shares for each representative and asset
df_shares_avg = (df_shares_high_end + df_shares_low_end) / 2

# Show the first few rows of one of the DataFrames to verify the calculations
df_shares_high_end.head(), df_shares_low_end.head(), df_shares_avg.head()



(                                   WFC          VMC            IR          KR  \
 A. Mitchell Jr. McConnell -6876.783713 -4052.019634 -14854.377868 -309.178209   
 Abigail Spanberger            0.000000     0.000000      0.000000    0.000000   
 Adam B. Schiff                0.000000     0.000000      0.000000    0.000000   
 Adam Kinzinger                0.000000     0.000000      0.000000    0.000000   
 Alan S. Lowenthal             0.000000     0.000000      0.000000    0.000000   
 
                                 LHX        ABBV          EGY         USFD  \
 A. Mitchell Jr. McConnell   0.00000    0.000000     0.000000     0.000000   
 Abigail Spanberger        -71.54807    0.000000     0.000000     0.000000   
 Adam B. Schiff              0.00000 -560.451452     0.000000     0.000000   
 Adam Kinzinger              0.00000    0.000000 -9232.702945 -7382.749903   
 Alan S. Lowenthal           0.00000    0.000000     0.000000     0.000000   
 
                               USO  

## Get Data Frame by Representative

In [38]:
from datetime import datetime

# Filter the data to only include transactions from January 1, 2018, onwards
df_main_filtered['TransactionDate'] = pd.to_datetime(df_main_filtered['TransactionDate'])
df_main_2018_onwards = df_main_filtered[df_main_filtered['TransactionDate'] >= datetime(2018, 1, 1)]

# Create a dictionary of DataFrames, one for each representative
dfs_by_representative = {}
for rep in unique_reps:
    dfs_by_representative[rep] = df_main_2018_onwards[df_main_2018_onwards['Representative'] == rep]

# Show the first few rows of one of the DataFrames as a sample
sample_rep = unique_reps[0]  # Take the first representative as a sample
dfs_by_representative[sample_rep].head()


,ReportDate,TransactionDate,Ticker,Representative,Transaction,Amount,Party,House,Range,Sentiment,Price,Low_End,High_End,Adjusted_Low_End,Adjusted_High_End,Running_Portfolio_Low_End,Running_Portfolio_High_End,Est_Portfolio_Value,Shares_High_End,Shares_Low_End
28054,2018-05-15,2018-03-05,WFC,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,49.059349,1001.0,15000.0,-1001.0,-15000.0,-3003.0,-45000.0,-24001.5,-305.752120,-20.403858
27828,2018-06-13,2018-06-01,WFC,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,46.939678,1001.0,15000.0,-1001.0,-15000.0,-4004.0,-60000.0,-32002.0,-319.559072,-21.325242
26691,2018-09-14,2018-09-06,WFC,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,50.084381,1001.0,15000.0,-1001.0,-15000.0,-5005.0,-75000.0,-40002.5,-299.494566,-19.986271
24113,2019-03-06,2019-03-05,WFC,A. Mitchell Jr. McConnell,Purchase,1001.0,R,Senate,1001 - 15000,NaN,43.876400,1001.0,15000.0,-1001.0,-15000.0,-7007.0,-105000.0,-56003.5,-341.869433,-22.814087
22252,2019-06-14,2019-06-03,VMC,A. Mitchell Jr. McConnell,Sale,250001.0,R,Senate,250001 - 500000,NaN,123.395256,250001.0,500000.0,250001.0,500000.0,241993.0,380000.0,310996.5,4052.019634,2026.017921


In [39]:
import os

# Change to the directory where you want to save the files
os.chdir('/Users/lukejohnson/Desktop/Congressional Trading/Trading History')

# Save each DataFrame as a separate CSV file
for rep in dfs_by_representative.keys():
    # Create a valid file name by replacing any characters that are not allowed in file names
    safe_rep_name = rep.replace(" ", "_").replace(".", "").replace(",", "").replace("'", "")
    dfs_by_representative[rep].to_csv(f'{safe_rep_name}_trading_history_2018_onwards.csv', index=False)


## Exploratory Review

Now lets review sentiment scores that were negative and purchases made at that time. As it would be the oppositie view point to buy these positions at this time.

In [40]:
# Ensure 'TransactionDate' and 'Sentiment' are in the correct format
df_main_filtered['TransactionDate'] = pd.to_datetime(df_main_filtered['TransactionDate'])
df_main_filtered['Sentiment'] = pd.to_numeric(df_main_filtered['Sentiment'], errors='coerce')

# Filter the data to include only 'Purchase' transactions
df_purchases = df_main_filtered[df_main_filtered['Transaction'] == 'Purchase']

# Filter the data to include only negative market sentiment
df_negative_sentiment = df_main_filtered[df_main_filtered['Sentiment'] < 0]

# Merge the two filtered DataFrames to find purchases made when market sentiment was negative
df_purchases_negative_sentiment = pd.merge(df_purchases, df_negative_sentiment, on=['TransactionDate', 'Ticker'], how='inner')

# Show the first few rows of the resulting DataFrame
df_purchases_negative_sentiment.head()


,ReportDate_x,TransactionDate,Ticker,Representative_x,Transaction_x,Amount_x,Party_x,House_x,Range_x,Sentiment_x,...,Price_y,Low_End_y,High_End_y,Adjusted_Low_End_y,Adjusted_High_End_y,Running_Portfolio_Low_End_y,Running_Portfolio_High_End_y,Est_Portfolio_Value_y,Shares_High_End_y,Shares_Low_End_y
0,2022-06-19,2022-06-15,BA,Alan S. Lowenthal,Purchase,1001.0,D,Representatives,1001-15000,-0.046329,...,133.720001,1001.0,15000.0,-1001.0,-15000.0,269239.09,1745000.0,1007119.545,-112.174692,-7.485791
1,2022-06-19,2022-06-15,BA,Alan S. Lowenthal,Purchase,1001.0,D,Representatives,1001-15000,-0.046329,...,133.720001,1001.0,15000.0,-1001.0,-15000.0,268238.09,1730000.0,999119.045,-112.174692,-7.485791
2,2022-06-19,2022-06-15,BA,Alan S. Lowenthal,Purchase,1001.0,D,Representatives,1001-15000,-0.046329,...,133.720001,1001.0,15000.0,-1001.0,-15000.0,269239.09,1745000.0,1007119.545,-112.174692,-7.485791
3,2022-06-19,2022-06-15,BA,Alan S. Lowenthal,Purchase,1001.0,D,Representatives,1001-15000,-0.046329,...,133.720001,1001.0,15000.0,-1001.0,-15000.0,268238.09,1730000.0,999119.045,-112.174692,-7.485791
4,2022-10-01,2022-09-22,CRM,Alan S. Lowenthal,Purchase,1001.0,D,Representatives,1001-15000,-0.147750,...,150.149994,1001.0,15000.0,-1001.0,-15000.0,249233.09,1620000.0,934616.545,-99.900104,-6.666667


In [41]:
# Count the total number of such trades
total_trades = len(df_purchases_negative_sentiment)

# Extract the 'Range' and sort to find the highest range trades
highest_range_trades = df_purchases_negative_sentiment.sort_values(by='Range_x', ascending=False)

# Show the summary
total_trades, highest_range_trades.head()

# Sort the DataFrame by 'Sentiment_x' to find the most negative sentiment and highest range trades
most_negative_sentiment_high_value_trade = df_purchases_negative_sentiment.sort_values(by=['Sentiment_x', 'Range_x'], ascending=[True, False])

# Show the trade with the most negative sentiment and highest range
most_negative_sentiment_high_value_trade.head()


,ReportDate_x,TransactionDate,Ticker,Representative_x,Transaction_x,Amount_x,Party_x,House_x,Range_x,Sentiment_x,...,Price_y,Low_End_y,High_End_y,Adjusted_Low_End_y,Adjusted_High_End_y,Running_Portfolio_Low_End_y,Running_Portfolio_High_End_y,Est_Portfolio_Value_y,Shares_High_End_y,Shares_Low_End_y
166,2023-04-20,2023-03-06,MS,Josh Gottheimer,Purchase,1001.0,D,Representatives,1001-15000,-0.9490,...,96.641426,1001.0,15000.0,-1001.0,-15000.0,-6818.0,2295000.0,1144091.0,-155.212941,-10.357877
290,2023-06-24,2023-05-22,NSA,Lois Frankel,Purchase,1001.0,D,Representatives,1001-15000,-0.9406,...,36.965889,1001.0,15000.0,-1001.0,-15000.0,27027.0,405000.0,216013.5,-405.779501,-27.079019
16,2022-02-08,2022-01-13,JEF,Cindy Axne,Purchase,1001.0,D,Representatives,1001-15000,-0.9011,...,34.201958,1001.0,15000.0,-1001.0,-15000.0,-47047.0,-705000.0,-376023.5,-438.571386,-29.267331
377,2022-09-15,2022-08-25,ARKK,Tommy Tuberville,Purchase,50001.0,R,Senate,50001 - 100000,-0.8813,...,45.779999,50001.0,100000.0,-50001.0,-100000.0,-847946.0,-1045000.0,-946473.0,-2184.360041,-1092.201864
68,2023-02-24,2023-01-31,VZ,Daniel Goldman,Purchase,1001.0,D,Representatives,1001-15000,-0.8625,...,40.174049,1001.0,15000.0,-1001.0,-15000.0,97978.0,-230000.0,-66011.0,-373.375356,-24.916582
